# State Vector Prediction in a GlueX Tracking Detector

Useful links
 - https://www.jlab.org/AI/quarterly_problem
 - https://halldweb.jlab.org/talks/ML_lunch/Sep2019/
 - http://www.dannowitz.co/ds-blog/2019/12/02/deep-recurrent-particle-tracking#commenting=

In [1]:
# Importing required libraries
import pandas as pd
import numpy as np

In [4]:
# Imaporting data

MLC_traning_inputs_df = pd.read_csv('MLchallenge2_training.csv')
MLC_traning_df = pd.read_csv('MLchallenge2_training.csv')
test_in_df = pd.read_csv('test_in.csv', on_bad_lines='skip')
test_prediction_df = pd.read_csv('test_prediction.csv')

In [26]:
MLC_traning_inputs_df.head()

,x,y,z,px,py,pz,x1,y1,z1,px1,...,z23,px23,py23,pz23,x24,y24,z24,px24,py24,pz24
0,0.877079,1.32218,65,-0.243980,-0.053204,2.414260,-10.66900,0.330138,176.944,-0.253523,...,341.28,-0.226149,0.106356,2.407600,-27.43270,5.06932,343.405,-0.226602,0.106052,2.407550
1,0.786361,-2.48294,65,0.103229,0.432216,2.592910,7.36649,15.502000,176.944,0.205638,...,341.28,0.326228,0.295249,2.581860,25.26860,37.60720,343.405,0.325574,0.295009,2.581950
2,-13.133900,-26.53090,65,0.064432,-0.020771,0.952952,-7.58617,-30.686700,176.944,0.026643,...,341.28,-0.015729,-0.047137,0.948236,-8.45915,-39.71580,343.405,-0.013650,-0.047591,0.948226
3,18.454200,2.80469,65,-0.019384,0.069384,1.832590,18.04330,6.797470,176.944,0.013039,...,341.28,0.039513,0.051584,1.824120,20.76920,11.70620,343.405,0.039183,0.052164,1.824100
4,15.552100,-19.19600,65,-0.009768,-0.010642,2.366080,15.06810,-19.750200,176.944,-0.014308,...,341.28,-0.013718,-0.002290,2.350870,13.65750,-20.62340,343.405,-0.014211,-0.002519,2.350850


In [27]:
MLC_traning_df.head()

,x,y,z,px,py,pz,x1,y1,z1,px1,...,z23,px23,py23,pz23,x24,y24,z24,px24,py24,pz24
0,0.877079,1.32218,65,-0.243980,-0.053204,2.414260,-10.66900,0.330138,176.944,-0.253523,...,341.28,-0.226149,0.106356,2.407600,-27.43270,5.06932,343.405,-0.226602,0.106052,2.407550
1,0.786361,-2.48294,65,0.103229,0.432216,2.592910,7.36649,15.502000,176.944,0.205638,...,341.28,0.326228,0.295249,2.581860,25.26860,37.60720,343.405,0.325574,0.295009,2.581950
2,-13.133900,-26.53090,65,0.064432,-0.020771,0.952952,-7.58617,-30.686700,176.944,0.026643,...,341.28,-0.015729,-0.047137,0.948236,-8.45915,-39.71580,343.405,-0.013650,-0.047591,0.948226
3,18.454200,2.80469,65,-0.019384,0.069384,1.832590,18.04330,6.797470,176.944,0.013039,...,341.28,0.039513,0.051584,1.824120,20.76920,11.70620,343.405,0.039183,0.052164,1.824100
4,15.552100,-19.19600,65,-0.009768,-0.010642,2.366080,15.06810,-19.750200,176.944,-0.014308,...,341.28,-0.013718,-0.002290,2.350870,13.65750,-20.62340,343.405,-0.014211,-0.002519,2.350850


In [5]:
test_in_df.head()

,0.877,1.322,65.000,-0.244,-0.053,2.414,-10.669,0.330,176.944,-0.254,...,-0.237,0.092,2.407,-22.920,3.053,296.228,-0.236,0.093,2.407.1,298.354
0,0.786,-2.483,65.0,0.103,0.432,2.593,7.366,15.502,176.944,0.206,...,0.310,0.320,2.581,19.381,32.055,296.228,0.312,0.318,2.581,298.354
1,1.962,2.314,65.0,-0.194,0.088,3.085,-4.705,6.134,176.944,-0.172,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-5.597,-21.536,65.0,0.241,0.371,3.249,3.536,-9.556,176.944,0.296,...,0.358,0.245,3.240,296.228,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.030,-0.421,65.0,-0.079,0.042,0.836,-7.952,8.029,176.944,-0.033,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13.917,-4.450,65.0,0.031,0.059,3.108,15.137,-2.579,176.944,0.039,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
test_prediction_df.head()

,-2.312394510175942841e+01,3.142886007367698742e+00,-2.355915925989915116e-01,9.161158105769111326e-02,2.413377136435135295e+00
0,19.633486,32.319292,0.314376,0.316425,2.592952
1,-8.308506,-39.299613,-0.020097,-0.051232,0.948906
2,19.918838,10.664617,0.038102,0.047740,1.864014
3,13.649239,-20.616935,-0.015548,0.001471,2.323953
4,-8.150528,8.982575,-0.155552,0.145865,3.019162
